# **Web scraping utilizando Selenium**
### **Disciplina**: Coleta, Preparação e Análise de Dados
### Prof. Lucas Pessutto


#### **Créditos**: Código baseado na implementação de Rodrigo C. Barros, Nathan S. Gavenski, Lucas Kupssinskü e Luan Garcia

Este tutorial mostra como utilizar a biblioteca Selenium em Python para realizar web scraping.

Como exemplo, vamos acessar o site do IMDB, realizar o scraping de algumas informações dos filmes presentes na lista de 100 filmes mais populares do site e depois armazenar essas informações em um arquivo JSON.

Para executar o código, garanta que você tenha uma versão do Selenium 4.6 ou mais nova.

Recomendo fazer a instalação em uma ambiente virtual conda, pois o versionamento dos drivers, Selenium e do Webdriver pode ser um pouco chato de lidar.

Pode acontecer de não encontrar uma versão 4.6 ou mais nova do Selenium utilizando apenas os respositórios do conda, então recomendo fazer a instalação do Selenium no ambiente através do pip:

```pip install selenium```

Maiores informacões:

Selenium web driver: [https://www.selenium.dev/documentation/webdriver/](https://www.selenium.dev/documentation/webdriver/)


Vamos importar as bibliotecas para trabalhar com o selenium. Precisamos importar um webdriver e o pacote ```By```, que utilizaremos a seguir para definir a forma de busca dos elementos web.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

Abaixo vamos inicializar uma sessão em um navegador Chrome utilizando o driver apropriado. Caso vá utilizar outro navegador, modifique o driver de acordo.

**Atenção**: A chamada abaixo primeiro procurar por um driver instalado localmente no PATH do sistema, depois por um driver na pasta raíz em que o script está sendo executado, e, só depois de não encontrar nenhuma dessas opções, ele procura automaticamente o driver certo para o navegador desejado. Recomendo não ter um driver no PATH ou na pasta raiz, pois qualquer atualização do seu navegador pode quebrar o código.

In [ ]:
driver = webdriver.Chrome()

Vamos abrir o site do IMDB no navegador e imprimir o título da página no notebook.

O comando ```get()``` abaixo acessa a página web localizada no link passado como parâmetro no navegador.

In [ ]:
driver.get("https://www.imdb.com")

In [ ]:
driver.maximize_window()

Agora que já carregamos o site podemos buscar pelos elementos HTML existentes na sessão atual do navegador utilizando a função ```find_element()```.

Lembrando que os elementos web no navegador são organizados em uma estrutura do tipo DOM (Document Object Modelo), que é o modelo que nos permite tratar um documento web html como se fosse uma árvore de objetos.

Mais infos sobre DOM: [https://javascript.info/dom-nodes](https://javascript.info/dom-nodes)

A função ```find_element()``` do Selenium encontra um elemento da árvore de acordo com algum tipo de localizador.
Os localizadores possíveis são:

```
    ID = busca pelo atributo id
    XPATH = busca pelo XPATH do elemento
    LINK_TEXT = busca pelo texto de um link
    PARTIAL_LINK_TEXT = busca pelo texto parcial de um link
    NAME = busca pelo atributo "name"
    TAG_NAME = busca pelo nome da tag
    CLASS_NAME = busca pelo nome do atributo classe
    CSS_SELECTOR = busca pelo nome do seletor CSS
```

Caso mais de um elemento seja encontrado, apenas o primeiro é retornado.

Lembrando que elementos web no modelo DOM podem possuir vários niveis de descendentes (são árvores), ou seja, o elemento pode possuir vários outros elementos dentro dele.

Como exemplo, vamos procurar o elemento web que contém o botão de abertura de menu no cabeçalho do site. Como estratégia de localização, vamos utilizar o endereço XPATH do elemento.

O próprio Chrome é bastante útil para encontrar o ID, Classe, XPATH ou qualquer outra forma que formos utilizar para selecionar um elemento web. Basta clicar com o botão direito no elemento que queremos e selecionar a opção *Inspecionar*.

Depois de acessar e inspecionar o site, é possível ver que o endereço XPATH do menu é o seguinte:
```//*[@id="imdbHeader-navDrawerOpen"]```

Lembrando que:
-  ```'//'``` indica que o endereço inicia da raíz do objeto DOM
- ```'*'``` indica que o elemento pode ter qualquer tag
- ```'[@id="imdbHeader-navDrawerOpen"]'``` indica que o elemento deve possuir o atributo *id* com o valor *imdbHeader-navDrawerOpen*.

Abaixo vamos executar a função ```find_element()```, indicando que queremos procurar o elemento pelo seu XPATH e vamos armazenar o resultado desta operação em uma variável chamada *menu*.

In [ ]:
menu = driver.find_element(By.XPATH, r"//*[@id='imdbHeader-navDrawerOpen']")

Por curiosidade, vamos imprimir o tipo de objeto retornado pela função ```find_element()```

In [ ]:
type(menu)

Podemos ver que o objeto retornado pela função ```find_element()``` é um objeto do tipo ```WebElement```, uma classe implementada pela biblioteca Selenium para representar um elemento web html.

Todo elemento web pode ser representado por uma tag html. Vamos ver abaixo qual o nome da tag html o site IMDB utiliza para representa o seu menu.

In [ ]:
menu.tag_name

Podemos ver que que o menu está sendo representado por uma tag do tipo *label*.

Agora, vamos pedir para o driver realizar uma ação de clique no navegador no elemento que selecionamos.

Podemos fazer isso chamando a função ```click()```. 

In [ ]:
menu.click()

A função ```click()``` acima fez com que o menu fosse aberto no site, e, com isso, o código javascript ser executado dinamicamente para gerar novos elementos web para criar os links no menu.

Podemos agora procurar pelo elemento que armazena o link para página de filmes mais populares que queremos acessar. Desta vez, vamos utilizar como estratégia de localização o texto presente no link.

Obs.: Tente executar o código abaixo sem que o menu esteja totalmente aberto e verifique o que acontece.

In [ ]:
link_populares = driver.find_element(By.LINK_TEXT, r'250 filmes mais populares')

Podemos ver abaixo que a função ```find_element()``` retornou novamente um objeto ```WebElement```, mas que agora que representa uma elemento do tipo *link* (TAG **a** do html).

In [ ]:
type(link_populares)

In [ ]:
link_populares.tag_name

Para descobrirmos a URL de fato, precisamos acessar o atributo *href* da tag a. Podemos fazer isto através da função ```get_attribute()```, passando como parâmetro o nome do atributo.

In [ ]:
url = link_populares.get_attribute("href")
url

Com o link correto em mãos, podemos novamente pedir para o driver realizar uma ação no navegador e acessar a página de filmes mais populares.

In [ ]:
driver.get(url)

Analizando a estrutura do código página (utilizando a função Inspecionar do Chrome), podemos ver que as informações dos filmes estão organizadas dentro de uma TAG html do tipo **ul** (unordered list), e cada filme está organizado dentro de uma tag **li**.

Vamos buscar a lista inteira utilizando seu caminho XPATH e depois todas suas tags *li*, para formar uma lista com os filmes.
Note que para as tags do tipo *li* vamos utilizar a função ```find_elements()``` (com s no final), que retorna uma lista de elementos.

In [ ]:
tag_ul = driver.find_element(By.XPATH, r'//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul')
lista_filmes = tag_ul.find_elements(By.TAG_NAME, "li")

Podemos conferir abaixo a diferença entre as funções ```find_element()``` e ```find_elements()```.

A primeira sempre retorna um único objeto de tipo ```WebElement```. A segunda sempre retorna uma lista, que pode estar vazia, conter apenas um objeto de tipo ```WebElement``` ou conter vários objetos de tipo ```WebElement```.

In [ ]:
type(tag_ul)

In [ ]:
type(lista_filmes)
lista_filmes

Vamos percorer a lista e imprimir o título dos filmes. Utilizando a inspeção do navegador é possível ver que o título fica armazenado em um elemento que tem o valor do atributo classe igual a ```"ipc-title__text"```.
Buscaremos o texto contido na tag utilizando como estratégia de localização o nome da classe do elemento.

In [ ]:
for filme in lista_filmes:
    print(filme.find_element(By.CLASS_NAME, "ipc-title__text").text)

Evidentemente nosso objetivo não é apenas imprimir os nomes dos filmes, mas sim recuperar diversas informações sobre cada filme e armazená-las de alguma forma.

Inicialmente, vamos definir uma classe ```Filme``` para armazenar todas informações que queremos de um filme.

Importaremos a biblioteca ```dataclasses``` para usar o decorador ```@dataclass```, que irá adicionar alguns métodos automaticamente, como o construtor de classe.

In [ ]:
from dataclasses import dataclass

@dataclass
class Filme:
    titulo: str
    ano: str
    classificacao: float
    pagina: str

Vamos definir uma função para que, dado um objeto ```WebElement``` que represente uma tag do tipo *li*, um objeto do tipo ```Filme``` seja criado e populado com as informações presentes nesta tag.

Analisando o site é possível ver que as tags que armazenam o ano dos filmes não possuem nenhuma classe ou atributo específico, então vamos precisar um XPATH como estratégio de localização.

Um ponto bastante relevante é que o XPATH neste caso precisa ser relativo à tag *li* que estamos acessando, pois se simplesmente copiarmos o XPATH completo de um elemento que contenha um ano específico, não iremos pegar corretamente os anos. Vamos ver a razão disto abaixo.

Observe que:

```//*[@id="__next"]/main/div/div[3]/section/div/div[2]/div/ul/li[2]/div[2]/div/div/div[3]/span[1]```

O XPATH acima inicia a partir da raiz da árvore e é específico para a segunda tag *li* da lista *ul* (a tag *li* está com o índice 2, ou seja .../ul/li[2]/div[2]/...):

Precisamos tornar este caminho relativo para que funcione em qualquer tag *li*:
```./div[2]/div/div/div[3]/span[1]```

O XPATH acima iniciando com o "." considera o endereço relativo do elemento, iniciando pelo endereço da tag *li* que estamos acessando e não pela raíz da árvore DOM.


Outro problema que temos é que o texto que contém o rating dos filmes está junto com o número de votos. 

Vamos precisar utilizar uma regex (expressão regular) para extrair apenas o valor da nota do filme.

In [ ]:
#biblioteca para expressões regulares
import re

def cria_filme(imdb_li_tag):
    titulo = imdb_li_tag.find_element(By.CLASS_NAME, "ipc-title__text").text #pegamos o texto do elemento
    ano = imdb_li_tag.find_element(By.XPATH, r'./div/div/div/div/div[2]/div[2]/span[1]').text 
    texto_classificacao = imdb_li_tag.find_element(By.CLASS_NAME, "ipc-rating-star").text #este texto está 'sujo', contém o número de votos também
    pagina = imdb_li_tag.find_element(By.CLASS_NAME, "ipc-title-link-wrapper").get_attribute("href")
    re_result = re.search(r'^\d\,\d',texto_classificacao) #aplicando a regex para extrair apenas o padrão 'digito ponto digito'
    
    classificacao = None
    if re_result:
        classificacao = re.search(r'^\d\,\d',texto_classificacao).group(0) #string resultante da regex
        classificacao = float(classificacao.replace(",", ".")) # Aletera o separador decimal
        
    return Filme(titulo, ano, classificacao, pagina)

**Atenção**: a função acima é bastante ingênua e assume que todas as tags vão estar corretas. Talvez algum filme possa não conter alguma informação. Para tornar o código mais robusto, utilize o comando try e trate exceções.

Exemplo:

'''
try:
    title = imdb_li_tag.find_element(By.CLASS_NAME, "ipc-title__text").text
except Exception as e:    
    print("Problema ao tentar ler dados do filme...")
'''

A lista dos possíveis erros pode ser encontrada na recomendação da W3C:

[https://www.w3.org/TR/webdriver/#errors](https://www.w3.org/TR/webdriver/#errors)

O selenium contém diversas classes de exceções implementadas para esses erros, verifique a documentação para utilizar a classe apropriada.

Vamos agora criar uma lista contendo todos os filmes que extraímos da página. Para isto, vamos utilizar uma estrutura de lista em Python, a classe ```Filme``` e o méotodo ```cria_filme``` que criamos.

In [ ]:
lista_de_filmes = []
for filme_tag in lista_filmes:
    filme = cria_filme(filme_tag)
    lista_de_filmes.append(filme)

Vamos imprimir os 10 primeiros filmes da lista:

In [ ]:
for i in range(10):
    print(lista_de_filmes[i])

Vamos agora armazenar nossa lista de filmes em um objeto JSON e salvar em um arquivo.

Precisaremos importar a biblioteca ```json``` e converter nosso objeto ```Filme``` para um dicionário, pois é o tipo de estrutura de dados aceita pela função ```dump()```.

O arquivo será salvo na mesma pasta em que o notebook está sendo executado.

In [ ]:
import json

with open("filmes.json", "w", encoding="utf-8") as arquivo:
    for filme in lista_de_filmes:
        json.dump(filme.__dict__, arquivo, ensure_ascii=False, indent=4) #.__dict__ converte um filme em uma estrutura do tipo Dicionário

Fechando o navegador

In [ ]:
driver.quit()